<a href="https://colab.research.google.com/github/mkufel/CryptoTrader/blob/master/CryptoTrader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.0.0
!pip install --upgrade keras
# Dataset https://www.kaggle.com/sudalairajkumar/cryptocurrencypricehistory#ethereum_price.csv

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    data_path = '/content/drive/My Drive/ID2223/crypto_data/'
    course_dir_path = '/content/drive/My Drive/ID2223'
except:
  print("Failed to connect to Drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from collections import deque
import random
from datetime import datetime
import time

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

In [0]:
currencies = ["bitcoin", "dash", "ethereum", "litecoin", "monero", "nem", "ripple"]

In [5]:
df_test = pd.read_csv(f"{data_path}bitcoin_price.csv", names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'])
df_test.head()

,Date,Open,High,Low,Close,Volume,Market Cap
0,Date,Open,High,Low,Close,Volume,Market Cap
1,"Feb 20, 2018",11231.80,11958.50,11231.80,11403.70,"9,926,540,000","189,536,000,000"
2,"Feb 19, 2018",10552.60,11273.80,10513.20,11225.30,"7,652,090,000","178,055,000,000"
3,"Feb 18, 2018",11123.40,11349.80,10326.00,10551.80,"8,744,010,000","187,663,000,000"
4,"Feb 17, 2018",10207.50,11139.50,10149.40,11112.70,"8,660,880,000","172,191,000,000"


In [6]:
# Get the date column for further joins, reverse the order
all_data = pd.DataFrame(data=df_test['Date'])[::-1]

for currency in currencies:
  # Read a file for the given currency
  df = pd.read_csv(f'{data_path}{currency}_price.csv', names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'])
  df = df.iloc[1:]

  # Rename columns adding currency names in headers
  df.rename(columns={
      'Open': f'Open_{currency}',
      'High': f'High_{currency}',
      'Low': f'Low_{currency}',
      'Close': f'Close_{currency}',
      'Volume': f'Volume_{currency}',
      'Market Cap': f'Market_cap_{currency}'},
      inplace=True)
  
  # Merge the renamed data frame with all_data on the Date column
  all_data = pd.merge(all_data, df, on='Date')

print(len(all_data))
print(all_data.head(5))

928
           Date Open_bitcoin  ... Volume_ripple Market_cap_ripple
0  Aug 08, 2015       279.74  ...       678,295       260,486,000
1  Aug 09, 2015       261.12  ...       531,969       270,702,000
2  Aug 10, 2015       265.48  ...       472,973       281,170,000
3  Aug 11, 2015       264.34  ...       282,461       279,220,000
4  Aug 12, 2015       270.60  ...       287,463       274,141,000

[5 rows x 43 columns]


In [0]:
TARGET_CURRENCY = 'monero'
SEQ_LEN_PAST = 14
SEQ_LEN_FUT = 1

In [8]:
all_data['Future'] = all_data[f"Close_{TARGET_CURRENCY}"].shift(-SEQ_LEN_FUT)
all_data.dropna(inplace=True)
all_data.tail()

,Date,Open_bitcoin,High_bitcoin,Low_bitcoin,Close_bitcoin,Volume_bitcoin,Market_cap_bitcoin,Open_dash,High_dash,Low_dash,Close_dash,Volume_dash,Market_cap_dash,Open_ethereum,High_ethereum,Low_ethereum,Close_ethereum,Volume_ethereum,Market_cap_ethereum,Open_litecoin,High_litecoin,Low_litecoin,Close_litecoin,Volume_litecoin,Market_cap_litecoin,Open_monero,High_monero,Low_monero,Close_monero,Volume_monero,Market_cap_monero,Open_nem,High_nem,Low_nem,Close_nem,Volume_nem,Market_cap_nem,Open_ripple,High_ripple,Low_ripple,Close_ripple,Volume_ripple,Market_cap_ripple,Future
922,"Feb 15, 2018",9488.32,10234.80,9395.58,10166.40,"9,062,540,000","160,025,000,000",679.79,705.67,667.21,703.86,"116,822,000","5,359,910,000",923.73,951.95,911.11,936.98,"2,900,100,000","90,173,300,000",212.35,237.88,211.17,225.43,"2,087,290,000","11,725,700,000",276.08,304.78,269.32,303.25,"85,772,700","4,340,620,000",0.569045,0.588881,0.544115,0.561359,"67,295,600","5,121,400,000",1.14,1.16,1.11,1.15,"1,188,530,000","44,433,600,000",296.48
923,"Feb 16, 2018",10135.70,10324.10,9824.82,10233.90,"7,296,160,000","170,960,000,000",702.53,722.02,684.28,698.86,"100,343,000","5,540,570,000",934.79,950.00,917.85,944.21,"2,369,450,000","91,272,100,000",225.50,236.09,212.33,230.12,"1,300,090,000","12,455,300,000",301.01,302.10,280.22,296.48,"64,320,200","4,733,640,000",0.560086,0.598202,0.548458,0.568887,"90,028,500","5,040,780,000",1.14,1.15,1.11,1.14,"568,339,000","44,663,300,000",325.67
924,"Feb 17, 2018",10207.50,11139.50,10149.40,11112.70,"8,660,880,000","172,191,000,000",699.13,748.89,693.64,742.16,"116,106,000","5,515,090,000",944.75,976.60,940.75,974.12,"2,525,720,000","92,264,000,000",229.50,233.45,225.60,229.47,"863,401,000","12,679,700,000",296.00,334.71,293.13,325.67,"134,269,000","4,655,820,000",0.565686,0.602952,0.560204,0.598880,"147,788,000","5,091,170,000",1.14,1.22,1.13,1.20,"1,186,730,000","44,370,200,000",300.12
925,"Feb 18, 2018",11123.40,11349.80,10326.00,10551.80,"8,744,010,000","187,663,000,000",744.09,747.42,673.57,689.28,"96,752,600","5,871,020,000",973.35,982.93,915.45,923.92,"2,567,290,000","95,077,100,000",230.51,232.64,214.46,215.74,"1,011,120,000","12,738,800,000",325.96,327.77,294.42,300.12,"68,135,100","5,128,420,000",0.595298,0.595298,0.527599,0.530407,"95,781,300","5,357,680,000",1.20,1.21,1.09,1.12,"1,113,650,000","46,997,500,000",316.49
926,"Feb 19, 2018",10552.60,11273.80,10513.20,11225.30,"7,652,090,000","178,055,000,000",687.70,731.11,687.22,728.73,"89,328,600","5,427,640,000",921.67,957.78,921.55,943.87,"2,169,020,000","90,047,700,000",215.41,226.97,215.38,223.06,"767,597,000","11,907,900,000",299.70,319.73,297.65,316.49,"67,482,400","4,716,460,000",0.528994,0.537727,0.518834,0.529261,"91,007,400","4,760,950,000",1.12,1.17,1.12,1.14,"556,304,000","43,813,300,000",304.40


In [9]:
# Add a column of target labels, if the rate rose in SEQ_LEN_FUT: 1, if fell: 0
all_data['Target'] = list(map(lambda past, future: int(float(future) > float(past)), all_data[f'Close_{TARGET_CURRENCY}'], all_data['Future']))
all_data[[f'Close_{TARGET_CURRENCY}', 'Future', 'Target']].head(5)

,Close_monero,Future,Target
0,0.676046,0.674739,0
1,0.674739,0.656576,0
2,0.656576,0.603582,0
3,0.603582,0.622923,1
4,0.622923,0.575855,0


In [10]:
def transform_date(date):
  return str(time.mktime(datetime.strptime(date, '%b %d, %Y').timetuple()))

all_data['Date'] = all_data['Date'].apply(transform_date)

all_data['Date'].head(5)

0    1438992000.0
1    1439078400.0
2    1439164800.0
3    1439251200.0
4    1439337600.0
Name: Date, dtype: object

In [11]:
cols = all_data.columns
all_data[cols] = all_data[cols].replace({',': ''}, regex=True)

all_data.head(1000)

,Date,Open_bitcoin,High_bitcoin,Low_bitcoin,Close_bitcoin,Volume_bitcoin,Market_cap_bitcoin,Open_dash,High_dash,Low_dash,Close_dash,Volume_dash,Market_cap_dash,Open_ethereum,High_ethereum,Low_ethereum,Close_ethereum,Volume_ethereum,Market_cap_ethereum,Open_litecoin,High_litecoin,Low_litecoin,Close_litecoin,Volume_litecoin,Market_cap_litecoin,Open_monero,High_monero,Low_monero,Close_monero,Volume_monero,Market_cap_monero,Open_nem,High_nem,Low_nem,Close_nem,Volume_nem,Market_cap_nem,Open_ripple,High_ripple,Low_ripple,Close_ripple,Volume_ripple,Market_cap_ripple,Future,Target
0,1438992000.0,279.74,279.93,260.71,261.00,58533000,4048270000,3.27,3.29,3.06,3.08,91264,18477800,2.79,2.80,0.714725,0.753325,674188,167911000,4.22,4.22,3.84,3.85,4917730,175126000,0.742898,0.744513,0.675775,0.676046,58680,6557890,0.000145,0.000147,0.000135,0.000136,3307,1304060,0.008164,0.008708,0.008164,0.008476,678295,260486000,0.674739,0
1,1439078400.0,261.12,267.00,260.47,265.08,23789600,3779770000,3.08,3.20,3.08,3.11,77488,17418300,0.706136,0.879810,0.629191,0.701897,532170,42637600,3.84,3.98,3.81,3.90,3064680,159570000,0.676069,0.696012,0.633000,0.674739,101826,5976810,0.000136,0.000140,0.000126,0.000130,8393,1223400,0.008484,0.008823,0.008472,0.008808,531969,270702000,0.656576,0
2,1439164800.0,265.48,267.03,262.60,264.47,20979400,3843850000,3.10,3.15,3.06,3.08,59702,17558200,0.713989,0.729854,0.636546,0.708448,405283,43130000,3.90,3.98,3.90,3.95,2239890,162062000,0.686737,0.698476,0.656262,0.656576,33307,6080090,0.000130,0.000139,0.000130,0.000135,1268,1165610,0.008812,0.008905,0.008746,0.008750,472973,281170000,0.603582,0
3,1439251200.0,264.34,270.39,264.09,270.39,25433900,3828500000,3.08,3.16,3.06,3.16,73134,17437300,0.708087,1.13,0.663235,1.07,1463100,42796500,3.95,4.16,3.94,4.16,3426300,164164000,0.656747,0.657478,0.599139,0.603582,69221,5823060,0.000135,0.000139,0.000126,0.000135,3006,1218210,0.008751,0.008766,0.008591,0.008591,282461,279220000,0.622923,1
4,1439337600.0,270.60,270.67,265.47,266.38,26815400,3920210000,3.16,3.18,3.06,3.06,113443,17915300,1.06,1.29,0.883608,1.22,2150620,64018400,4.14,4.16,3.97,3.99,3519900,172371000,0.603783,0.651037,0.602168,0.622923,51925,5361380,0.000135,0.000138,0.000128,0.000133,5295,1213900,0.008591,0.008593,0.008271,0.008271,287463,274141000,0.575855,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,1518652800.0,9488.32,10234.80,9395.58,10166.40,9062540000,160025000000,679.79,705.67,667.21,703.86,116822000,5359910000,923.73,951.95,911.11,936.98,2900100000,90173300000,212.35,237.88,211.17,225.43,2087290000,11725700000,276.08,304.78,269.32,303.25,85772700,4340620000,0.569045,0.588881,0.544115,0.561359,67295600,5121400000,1.14,1.16,1.11,1.15,1188530000,44433600000,296.48,0
923,1518739200.0,10135.70,10324.10,9824.82,10233.90,7296160000,170960000000,702.53,722.02,684.28,698.86,100343000,5540570000,934.79,950.00,917.85,944.21,2369450000,91272100000,225.50,236.09,212.33,230.12,1300090000,12455300000,301.01,302.10,280.22,296.48,64320200,4733640000,0.560086,0.598202,0.548458,0.568887,90028500,5040780000,1.14,1.15,1.11,1.14,568339000,44663300000,325.67,1
924,1518825600.0,10207.50,11139.50,10149.40,11112.70,8660880000,172191000000,699.13,748.89,693.64,742.16,116106000,5515090000,944.75,976.60,940.75,974.12,2525720000,92264000000,229.50,233.45,225.60,229.47,863401000,12679700000,296.00,334.71,293.13,325.67,134269000,4655820000,0.565686,0.602952,0.560204,0.598880,147788000,5091170000,1.14,1.22,1.13,1.20,1186730000,44370200000,300.12,0
925,1518912000.0,11123.40,11349.80,10326.00,10551.80,8744010000,187663000000,744.09,747.42,673.57,689.28,96752600,5871020000,973.35,982.93,915.45,923.92,2567290000,95077100000,230.51,232.64,214.46,215.74,1011120000,12738800000,325.96,327.77,294.42,300.12,68135100,5128420000,0.595298,0.595298,0.527599,0.530407,95781300,5357680000,1.20,1.21,1.09,1.12,1113

In [12]:
# Get the date boundary of the last 10% of records - test_set and 25% of records - 25%-10% - val_set
last_10pc = all_data['Date'][int(0.9*len(all_data.index.values))]
last_25pc = all_data['Date'][int(0.75*len(all_data.index.values))]

print('Last 10% starts from date: {}'.format(datetime.fromtimestamp(float(last_10pc)).strftime('%Y-%m-%d')))
print('Last 25% starts from date: {}'.format(datetime.fromtimestamp(float(last_25pc)).strftime('%Y-%m-%d')))

Last 10% starts from date: 2017-11-19
Last 25% starts from date: 2017-07-03


In [13]:
# Split the input dataframe into test (last 10%), validation (subset(75%-90%)), test(first 75%) sets
test_df = all_data[(all_data['Date'] >= last_10pc)]
val_df = all_data[(all_data['Date'] >= last_25pc)][all_data['Date'] < last_10pc]
train_df = all_data[(all_data['Date'] < last_25pc)]

print(f"Training set shape: {train_df.shape}")
print(f"Validation set shape: {val_df.shape}")
print(f"Test set shape: {test_df.shape}")

Training set shape: (695, 45)
Validation set shape: (139, 45)
Test set shape: (93, 45)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [0]:
# Scale the data and organize in sequences of length SEQ_LEN_PAST
def preprocess(df, isTest=False):
  df.drop('Future', 1)
  
  dates_scaled_dict = {}

  # Scale all columns except for target
  for c in df.columns:
    if c != 'Target':
      temp_pct = df[c].astype(float).pct_change()
      temp_scaled = preprocessing.scale(temp_pct.values)

      # Map scaled dates to real dates to interpret test results
      if isTest and c == 'Date':
        dates_scaled_dict = dict(zip(temp_scaled, df[c].values))

      df[c] = temp_scaled

  df.dropna(inplace=True)

  # Generate sequences
  sequences = []
  past_seq = deque(maxlen=SEQ_LEN_PAST)

  for row_values in df.values:
    # append all values in the row except for target into the past_seq
    past_seq.append([n for n in row_values[:-1]])
    
    # if past_seq of the desired length, appens to sequences
    if len(past_seq) == SEQ_LEN_PAST:
      sequences.append([np.array(past_seq), row_values[-1]])


  # if processing a test set, skip balancing
  if isTest:
    x = []
    for s, t in sequences:
      x.append(s)

    return np.array(x), dates_scaled_dict
  
  random.shuffle(sequences)

  # Balance the data - make sure there are as many rise and fall sequences
  ups = []
  downs = []

  for seq, target in sequences:
    if target:
      ups.append([seq, target])
    else:
      downs.append([seq, target])

  lower = min(len(ups), len(downs))
  balanced_sequences = ups[:lower] + downs[:lower]
  
  features = []
  targets = []
  
  for seq, target in balanced_sequences:
    features.append(seq)
    targets.append(target)
  
  return np.array(features), np.array(targets)

In [15]:
pd.set_option('display.expand_frame_repr', False)

features_train, labels_train = preprocess(train_df.copy())
features_val, labels_val = preprocess(val_df.copy())
features_test, dates_scaled_dict = preprocess(test_df.copy(), True)

print(f'Training features length: {len(features_train)}, labels length {len(labels_train)}')
print(f'Validation features length: {len(features_val)}, labels length {len(labels_val)}')
print(f'Test features length: {len(features_test)}')

Training features length: 674, labels length 674
Validation features length: 110, labels length 110
Test features length: 79


In [0]:
EPOCHS = 13
BATCH_SIZE = 4

In [21]:
model = Sequential()

model.add(LSTM(256, input_shape=(features_train.shape[1:]), return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(LSTM(256, input_shape=(features_train.shape[1:]), return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(LSTM(256, input_shape=(features_train.shape[1:])))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(2, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 14, 256)           308224    
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 256)           0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 14, 256)           1024      
_________________________________________________________________
lstm_4 (LSTM)                (None, 14, 256)           525312    
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 256)           0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 256)           1024      
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)              

In [17]:
model.fit(features_train, labels_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(features_val, labels_val))

Train on 674 samples, validate on 110 samples
Epoch 1/13
674/674 [==============================] - 38s 56ms/sample - loss: 1.0019 - accuracy: 0.5193 - val_loss: 1.1197 - val_accuracy: 0.4545
Epoch 2/13
674/674 [==============================] - 31s 46ms/sample - loss: 0.9826 - accuracy: 0.4599 - val_loss: 0.7809 - val_accuracy: 0.5000
Epoch 3/13
674/674 [==============================] - 31s 46ms/sample - loss: 0.9193 - accuracy: 0.4540 - val_loss: 0.8677 - val_accuracy: 0.4273
Epoch 4/13
674/674 [==============================] - 31s 47ms/sample - loss: 0.7998 - accuracy: 0.4881 - val_loss: 0.7321 - val_accuracy: 0.5091
Epoch 5/13
674/674 [==============================] - 31s 46ms/sample - loss: 0.7553 - accuracy: 0.5193 - val_loss: 0.7515 - val_accuracy: 0.5182
Epoch 6/13
674/674 [==============================] - 32s 47ms/sample - loss: 0.7801 - accuracy: 0.4763 - val_loss: 0.7002 - val_accuracy: 0.4909
Epoch 7/13
674/674 [==============================] - 32s 48ms/sample - loss: 

In [0]:
pred = model.predict(features_test)

In [19]:
y_true = []
y_pred = []

# Print dates of the input test sequence and print prediction of the prices going up or down
for idx, seq in enumerate(features_test): 
  for day in seq:
    date = datetime.fromtimestamp(float(dates_scaled_dict[day[0]])).strftime('%d-%m-%Y')
    print('{}, '.format(date), end = '')

  # Append corresponding target(true) values form test_df to y_true  
  true_value = test_df.loc[test_df['Date'] == dates_scaled_dict[seq[0][0]]]['Target'].values[0]
  y_true.append(true_value)

  will_rise = 0
  if pred[idx][1] > 0.5: 
    will_rise = 1

  y_pred.append(will_rise)  

  print(f'Predicted: {will_rise}, Real: {true_value}')

20-11-2017, 21-11-2017, 22-11-2017, 23-11-2017, 24-11-2017, 25-11-2017, 26-11-2017, 27-11-2017, 28-11-2017, 29-11-2017, 30-11-2017, 01-12-2017, 02-12-2017, 03-12-2017, Predicted: 1, Real: 1
21-11-2017, 22-11-2017, 23-11-2017, 24-11-2017, 25-11-2017, 26-11-2017, 27-11-2017, 28-11-2017, 29-11-2017, 30-11-2017, 01-12-2017, 02-12-2017, 03-12-2017, 04-12-2017, Predicted: 1, Real: 1
22-11-2017, 23-11-2017, 24-11-2017, 25-11-2017, 26-11-2017, 27-11-2017, 28-11-2017, 29-11-2017, 30-11-2017, 01-12-2017, 02-12-2017, 03-12-2017, 04-12-2017, 05-12-2017, Predicted: 1, Real: 0
23-11-2017, 24-11-2017, 25-11-2017, 26-11-2017, 27-11-2017, 28-11-2017, 29-11-2017, 30-11-2017, 01-12-2017, 02-12-2017, 03-12-2017, 04-12-2017, 05-12-2017, 06-12-2017, Predicted: 1, Real: 1
24-11-2017, 25-11-2017, 26-11-2017, 27-11-2017, 28-11-2017, 29-11-2017, 30-11-2017, 01-12-2017, 02-12-2017, 03-12-2017, 04-12-2017, 05-12-2017, 06-12-2017, 07-12-2017, Predicted: 1, Real: 1
25-11-2017, 26-11-2017, 27-11-2017, 28-11-2017, 29

In [20]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true, y_pred)

0.6455696202531646